In [ ]:
import pandas as pd
import numpy as np
import json
import re
import os
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import sklearn
#from sklearn import *
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from nltk.tokenize import TweetTokenizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV


import joblib
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

#import tensorflow as tf
#from keras.models import Sequential
#from keras.layers import Dense
#from keras.layers import Dropout
#from keras.layers import Flatten
#from keras.metrics import *
#import keras

In [ ]:
import xgboost as xgb

In [ ]:
from sklearn.inspection import permutation_importance
import rfpimp
#from eli5.sklearn import PermutationImportance

In [ ]:
import pandas as pd

In [ ]:
%matplotlib inline

In [ ]:
p_in="/home/lindali/Dropbox/bot and agenda-setting/codes and data/bot_identification_code_and_data/"

In [ ]:
#pd.read_json(p_in+"training_set/Twibot-20/dev.json")

### Data preprocessing

In [ ]:
# Reading in json training set & data preprocessing
def json_unifier(file_path, is_bot = None):
    with open(file_path) as f:
        file = json.loads(f.read())
        key_to_extract = list(file[0].keys())[1]
        user_dict_list = [i[key_to_extract] for i in file]
    df = pd.DataFrame(user_dict_list)
    if is_bot is not None:
        df['is_bot'] = is_bot
    elif file_path != "./training_set/Twibot-20/train.json":
        identification_path = file_path.replace("_tweets.json", ".tsv")
        tag_df = pd.read_csv(identification_path, sep='\t', names = ['id','is_bot'])
        tag_df['is_bot'] = [1.0 if i == "bot" else 0.0 for i in list(tag_df['is_bot'])]
        df = pd.merge(df,tag_df,left_on='id',right_on='id', copy = False, suffixes=(None, '_y'))
        #df['is_bot'] = df["is_bot_y"]
        #df.drop(labels = ["id_y", "is_bot_y"], axis = 1)
        #print(df)
    else:
        with open(file_path) as f:
            file = json.loads(f.read())
            key_to_extract_new = list(file[0].keys())[5]
            is_bot = [int(i[key_to_extract_new]) for i in file]
        df['is_bot'] = is_bot
    return df

training_set_paths = {p_in+"training_set/Twibot-20/train.json":None,
                     p_in+"training_set/verified-2019_tweets.json":0,
                     p_in+"training_set/vendor-purchased-2019_tweets.json":1,
                     p_in+"training_set/political-bots-2019_tweets.json":1,
                     p_in+"training_set/cresci-rtbust-2019_tweets.json":None,
                      p_in+"training_set/botometer-feedback-2019_tweets.json":None,
                     p_in+"training_set/botwiki-2019/botwiki-2019_tweets.json":1
                     }
all_df = pd.DataFrame()
for file,is_bot in training_set_paths.items():
    #print(is_bot)
    training_df = json_unifier(file, is_bot)
    #print(training_df)
    all_df = pd.concat([all_df, training_df], ignore_index=True)
    #all_df = all_df.append(training_df,ignore_index = True)


In [ ]:
file_path = "./training_set/Twibot-20/train.json"
is_bot = None

In [ ]:
# read in csv files
delete_list = ['timestamp', 'crawled_at', 'updated', 'test_set_1','test_set_2']
def csv_unifier(file_path, is_bot = None):
    df = pd.read_csv(file_path)
    column_list = list(df.columns)
    column_list = [i for i in column_list if i not in delete_list]
    df = df[column_list]
    if is_bot is not None:
        df['is_bot'] = is_bot
    return df

csv_set_paths = {"./training_set/users_real.csv":0,
            "./training_set/users.csv":1,
            "./training_set/users_fake_follower.csv":1,
                 "./training_set/users_socialbot.csv":1
                }
for file, is_bot in csv_set_paths.items():
    training_df = csv_unifier(file, is_bot)
    all_df = pd.concat([all_df, training_df], ignore_index=True)


In [ ]:
def generate_features(all_df, create_param = 'created_at'):
    created_at_dt = pd.to_datetime(all_df[create_param], format = "%a %b %d %H:%M:%S +0000 %Y ", errors = 'coerce')
    created_at_dt_new = pd.to_datetime(all_df[create_param], format = "%a %b %d %H:%M:%S +0000 %Y", errors = 'coerce')
    created_at_dt = [created_at_dt_new[i] if pd.isnull(created_at_dt[i]) else created_at_dt[i] for i in range(len(created_at_dt))]
    all_df['created_at_dt'] = created_at_dt
    all_df['user_age'] = [datetime.now() - i for i in all_df['created_at_dt']]
    all_df['user_age'] = [i.total_seconds()//3600 for i in all_df['user_age']]
    main_traits = ['statuses_count', 'followers_count', 'friends_count', 'favourites_count', 'listed_count']
    derived_traits = ['tweet_freq','followers_growth','friends_growth', 'favourites_growth', 'listed_growth']

    for i in range(5):
        all_df[main_traits[i]] = [int(k) for k in all_df[main_traits[i]]]
        all_df[derived_traits[i]] = all_df[main_traits[i]]/all_df['user_age']

    all_df['follower_friend_ratio'] = all_df['followers_count']/(all_df['friends_count'] + 1)
    all_df['screen_name_length'] = [len(i) for i in all_df['screen_name']]
    all_df['digits_in_screen_name'] = [sum(c.isdigit() for c in i) for i in all_df['screen_name']]
    all_df['name_length'] = [len(i) if pd.notnull(i) else 0 for i in all_df['name']]
    all_df['digits_in_name'] = [sum(c.isdigit() for c in i) if pd.notnull(i) else 0 for i in all_df['name']]
    all_df['description_length'] = [len(i) if pd.notnull(i) else 0 for i in all_df['description']]
    return all_df

In [ ]:
all_df = generate_features(all_df)

In [ ]:
all_df.tail()

In [ ]:
len(all_df)

In [ ]:
# I have much smaller sample Counter({0.0: 9813, 1.0: 14785})

In [ ]:
from collections import Counter
Counter(all_df.is_bot)

In [ ]:
all_df['user_age'] = [datetime.now() - i for i in all_df['created_at_dt']]
all_df['user_age'] = [i.total_seconds()//3600 for i in all_df['user_age']]
main_traits = ['statuses_count', 'followers_count', 'friends_count', 'favourites_count', 'listed_count']
derived_traits = ['tweet_freq','followers_growth','friends_growth', 'favourites_growth', 'listed_growth']

for i in range(5):
    all_df[main_traits[i]] = [int(k) for k in all_df[main_traits[i]]]
    all_df[derived_traits[i]] = all_df[main_traits[i]]/all_df['user_age']

all_df['follower_friend_ratio'] = all_df['followers_count']/(all_df['friends_count'] + 1)
all_df['screen_name_length'] = [len(i) for i in all_df['screen_name']]
all_df['digits_in_screen_name'] = [sum(c.isdigit() for c in i) for i in all_df['screen_name']]
all_df['name_length'] = [len(i) if pd.notnull(i) else 0 for i in all_df['name']]
all_df['digits_in_name'] = [sum(c.isdigit() for c in i) if pd.notnull(i) else 0 for i in all_df['name']]
all_df['description_length'] = [len(i) if pd.notnull(i) else 0 for i in all_df['description']]

In [ ]:
# (1) statuses count; (2) followers count; (3)
#friends count; (4) favorites count; (5) listed count; (6) default profile; (7) geo enabled; (8) profile
#use background image; (9) protected; and (10) verified. 

def get_train_test(df, test_size = 0.3, random_state = 15):
    features = ['statuses_count', 'followers_count', "friends_count", "favourites_count","listed_count","default_profile",
               "geo_enabled","profile_use_background_image",
                "protected", 
                "verified",'tweet_freq','followers_growth','friends_growth', 'favourites_growth', 'listed_growth',
               'follower_friend_ratio', 'screen_name_length','digits_in_screen_name', 
               'name_length','digits_in_name', 'description_length']

    boolean_features = ["default_profile","geo_enabled","profile_use_background_image", "verified",
                       "protected"]
    #boolean_features = ["protected", "verified"]
    x_df = df[features]
    x_df = x_df.fillna(0.0)
    x_df[boolean_features] = x_df[boolean_features].replace("True ",1.0)
    x_df[boolean_features] = x_df[boolean_features].replace("False ",0.0)
    x_df = x_df.apply(lambda x: np.asarray(x).astype(np.float32))

    y_df = df['is_bot']
    xtrain, xtest, ytrain, ytest = train_test_split(x_df, y_df, test_size = test_size, random_state = random_state)
    return xtrain, xtest, ytrain, ytest

xtrain, xtest, ytrain, ytest = get_train_test(all_df)

### Random forest

In [ ]:
# baseline: botometre
rf_cols = ['statuses_count', 'followers_count', "friends_count", "favourites_count","listed_count","default_profile",
               "geo_enabled","profile_use_background_image",
                "protected", 
                "verified"]
xtest_rf = xtest[rf_cols]
xtrain_rf = xtrain[rf_cols]




In [ ]:
rf_test1 = {'n_estimators':[i for i in range(10,201,10)]}
rf_search1 = GridSearchCV(estimator = RandomForestClassifier(min_samples_split=100,
                                  max_depth=8,max_features='sqrt' ,random_state=10), 
                       param_grid = rf_test1, scoring='roc_auc',cv=5)
rf_search1.fit(xtrain_rf,ytrain)
rf_search1.cv_results_


In [ ]:
#RandomForestClassifier(max_depth=8, min_samples_split=100, n_estimators=180,
#                       random_state=10)

rf_search1.best_estimator_

In [ ]:
rf_search1.best_estimator_

In [ ]:
rf_search1.best_estimator_

In [ ]:
# Random forest
forest = RandomForestClassifier(n_estimators=150,max_depth=18, max_features='sqrt' ,random_state=10)
forest.fit(xtrain,ytrain)
forest_result = list(forest.predict(xtest))

In [ ]:
print(sklearn.metrics.classification_report(ytest, forest_result))

In [ ]:
#print(sklearn.metrics.classification_report(ytest, forest_result))

In [ ]:
forest_10 = RandomForestClassifier(n_estimators=150,max_depth=18, max_features='sqrt' ,random_state=10)
forest_10.fit(xtrain_rf,ytrain)
forest_10_result = list(forest_10.predict(xtest_rf))

In [ ]:
print(sklearn.metrics.classification_report(ytest, forest_10_result))

In [ ]:
print(sklearn.metrics.classification_report(ytest, forest_10_result))

In [ ]:
lr_probs = forest.predict_proba(xtest_rf)
lr_probs = lr_probs[:, 1]
lr_auc = roc_auc_score(list(ytest), lr_probs)
lr_fpr, lr_tpr, _ = roc_curve(list(ytest), lr_probs)
plt.plot(lr_fpr, lr_tpr)

In [ ]:
# Logistic regression
logit = LogisticRegression()
logit.fit(xtrain_rf,ytrain)
logit_result = list(logit.predict(xtest_rf))

In [ ]:
print(sklearn.metrics.classification_report(ytest,logit_result))

In [ ]:
joblib.dump(forest, 'random_forest_model_20trait.joblib') 
joblib.dump(forest_10, 'random_forest_model_20trait.joblib') 

In [ ]:
#Visualization of metrics
# accuracy, percision, recall, f1, sensivity, specificity
forest_metric = sklearn.metrics.classification_report(ytest, forest_result,output_dict=True)
svm_metric = sklearn.metrics.classification_report(ytest,svm_result,output_dict=True)
logit_metric = sklearn.metrics.classification_report(ytest,logit_result,output_dict=True)


In [ ]:
print(forest_metric)
print(svm_metric)
print(logit_metric)

In [ ]:
method = ['RF',"SVM","LOGR"]
df_metric = pd.DataFrame()
for index, metrics in enumerate([forest_metric, svm_metric, logit_metric]):
    df = pd.DataFrame(metrics).transpose()
    df_temp = pd.DataFrame()
    df_temp["value"] = df['recall'].iloc[:4]
    df_temp['metric'] = ['Specificity','Sensitivity','Balanced Accuracy','Accuracy']
    df_temp = df_temp.append({"metric":"F1", "value": float(df.iloc[4,2])}, ignore_index = True)
    df_temp['method'] = method[index]
    df_metric = df_metric.append(df_temp, ignore_index = True)

In [ ]:
df_metric

In [ ]:
dl_metric = pd.DataFrame({"metric": ['Specificity','Sensitivity','Balanced Accuracy','Accuracy','F1'],
                         "value":[0.8954,0.8529,0.8911,0.7718, 0.8719]}) # This was what I had before 
dl_metric['method'] = 'DL'
df_metric = df_metric.append(dl_metric, ignore_index = True)

In [ ]:
sns.set_style("ticks")

plt.figure(figsize=(10, 3.5))
ax = sns.barplot(data=df_metric, x='metric', y='value', hue='method',palette=("Paired"))

def change_width(ax, new_value) :
    for patch in ax.patches :
        current_width = patch.get_width()
        diff = current_width - new_value

        # we change the bar width
        patch.set_width(new_value)

        # we recenter the bar
        patch.set_x(patch.get_x() + diff * .5)

change_width(ax, .18)

plt.xlabel("")
plt.ylabel("Value",fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid()
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()
#plt.show()
#plt.savefig("metric_comparsion_updated_20.png")

## XGboost

In [ ]:
xgb_cl = xgb.XGBClassifier(booster= "gbtree")
eval_set = [(xtrain, ytrain), (xtest, ytest)]
eval_metric = ["auc","error"]


In [ ]:
xgb_cl.fit(xtrain, ytrain)
ypred = xgb_cl.predict(xtest)

In [ ]:
print(sklearn.metrics.classification_report(ytest, [1 if i >=0.5 else 0 for i in ypred]))

In [ ]:
params = {
 "learning_rate" : [0.05,0.10,0.15,0.20,0.25,0.30],
 "max_depth" : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma": [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
}

xgb_tuning = RandomizedSearchCV(xgb_cl, param_distributions=params, n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5, return_train_score=True)
search = xgb_tuning.fit(xtrain, ytrain)

In [ ]:
search.best_estimator_

In [ ]:
xgb_cl = xgb.XGBClassifier( learning_rate=0.15,n_estimators=100, colsample_bytree=0.3,gamma=0.4)
xgb_cl.fit(xtrain, ytrain)
ypred = xgb_cl.predict(xtest)

In [ ]:
print(sklearn.metrics.classification_report(ytest, ypred))

In [ ]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(xgb_cl,xtrain,ytrain,cv=10)

In [ ]:
score

In [ ]:
import joblib
joblib.dump(xgb_cl,  "/home/lindali/Documents/DPhil studies/thesis_work/bot_identification/xgboost_100623_noseed.joblib") 

### Testing

In [ ]:
#Validating with other training sets
def json_unifier(file_path, is_bot = None):
    with open(file_path) as f:
        file = json.loads(f.read())
    df = pd.DataFrame(file)
    #print(df)
    identification_path = file_path.replace("_tweets.json", ".tsv")
    tag_df = pd.read_csv(identification_path, sep='\t', names = ['id','is_bot'])
    tag_df['is_bot'] = [1.0 if i == "bot" else 0.0 for i in list(tag_df['is_bot'])]
    df = pd.merge(df,tag_df,left_on='user_id',right_on='id', copy = False, suffixes=(None, '_y'))
        #df['is_bot'] = df["is_bot_y"]
        #df.drop(labels = ["id_y", "is_bot_y"], axis = 1)
        #print(df)
    return df


In [ ]:
Counter(midterm_df.is_bot)

In [ ]:
midterm_df = json_unifier("/home/lindali/Documents/DPhil studies/thesis_work/bot_identification/training_set/midterm-2018/midterm-2018_tweets.json")
midterm_df

In [ ]:
midterm_df = json_unifier("/home/lindali/Documents/DPhil studies/thesis_work/bot_identification/training_set/midterm-2018/midterm-2018_tweets.json")
midterm_df = generate_features(midterm_df, 'user_created_at')
xtrain_us, xtest_us, ytrain_us, ytest_us = get_train_test(midterm_df,test_size = 0.3)

In [ ]:
len(midterm_df[midterm_df.is_bot==0])

In [ ]:
forest_result_us = list(forest.predict(xtrain_us))

In [ ]:
len([i for i in ytrain_us if i == 0])

In [ ]:
len([i for i in forest_result_us if i == 1])

In [ ]:
print(sklearn.metrics.classification_report(ytrain_us, forest_result_us))

In [ ]:
xgboost_result_us = list(xgb_cl.predict(xtrain_us))

In [ ]:
print(sklearn.metrics.classification_report(ytrain_us, xgboost_result_us))

In [ ]:
print(0.8492*0.9767*2/(0.8492+0.9767))

In [ ]:
us_metric = pd.DataFrame({"metric": ['Specificity','Sensitivity','Precision','Balanced Accuracy','Accuracy','F1'],
                         "value":[0.93,0.85,0.98, 0.83,0.91, 0.88]})
us_metric_rf = pd.DataFrame({"metric": ['Specificity','Sensitivity','Precision','Balanced Accuracy','Accuracy','F1'],
                         "value":[0.89,0.93,0.97, 0.86,0.86,0.90]})
us_metric['method'] = 'RF'
us_metric_rf['method'] = 'DL'
us_metric = us_metric.append(us_metric_rf, ignore_index = True)

In [ ]:
sns.set_style("ticks")

plt.figure(figsize=(10, 3.5))
ax = sns.barplot(data=us_metric, x='metric', y='value', hue='method')



plt.xlabel("")
plt.ylabel("Value",fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid()
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()
plt.savefig("metric_comparsion_rf_dl_updated.png")

In [ ]:
def json_unifier(file_path, is_bot = None):
    with open(file_path) as f:
        file = json.loads(f.read())
        key_to_extract = list(file[0].keys())[1]
        user_dict_list = [i[key_to_extract] for i in file]
    df = pd.DataFrame(user_dict_list)

    with open(file_path) as f:
        file = json.loads(f.read())
        key_to_extract_new = list(file[0].keys())[5]
        is_bot = [int(i[key_to_extract_new]) for i in file]
    df['is_bot'] = is_bot
    return df

In [ ]:
testing_df = json_unifier("./training_set/Twibot-20/test.json")

In [ ]:
xtrain_twi, xtest_twi, ytrain_twi, ytest_twi = get_train_test(midterm_df,test_size = 0.01)

In [ ]:
forest_result_twi = list(forest.predict(xtrain_twi))
print(sklearn.metrics.classification_report(ytrain_twi, forest_result_twi))

In [ ]:
model.evaluate(xtrain_twi, ytrain_twi, batch_size = 128)